<a href="https://colab.research.google.com/github/zakstucke/geo-sentiment-analysis/blob/master/SimpleSentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment analysis tools


In [ ]:
## Test notebook

# import pandas
import nltk

# VADER = Valence Aware Dictionary for Sentiment Reasoning
# Gives polarity and intensity of sentiment
nltk.download('vader_lexicon')

# Right from the documentation https://www.nltk.org/howto/sentiment.html
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
n_instances = 100
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]
len(subj_docs), len(obj_docs)

In [ ]:
# Feedback on a simple list of sentences
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentences = [
             "Covid is a good thing",
             "Covid is a bad thing"
             ]
pss = []   # polarity score's
for sentence in sentences:
  sia = SentimentIntensityAnalyzer()
  pss.append(sia.polarity_scores(sentence))

for ps in pss:
  print(f"neg: {ps['neg']}, pos: {ps['pos']}, neu: {ps['neu']}, cmp: {ps['compound']}")

neg: 0.0, pos: 0.492, neu: 0.508, cmp: 0.4404
neg: 0.538, pos: 0.0, neu: 0.462, cmp: -0.5423


In [ ]:
# Converting to dataframe for small scale processing
import pandas as pd
df = pd.DataFrame(pss)
print(df)

# Select confidently negative answers
negatives = df[(df['neg']>0.5) & (df['pos']<0.2)]
# Print confidently negative sentences
print(sentences[negatives.index[0]])

     neg    neu    pos  compound
0  0.000  0.508  0.492    0.4404
1  0.538  0.462  0.000   -0.5423
Covid is a bad thing


In [ ]:
!pip install nrclex
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nrclex import NRCLex

nltk.download("punkt")
nltk.download('vader_lexicon')

def create_emotions(df, text_column_name):
    emotions = ["emotion_1", "emotion_2", "emotion_3"]
    
    # Add in the new emotion columns:
    for emotion in emotions:
        df[emotion] = np.nan

    def analyze_emotions(row):
        # Analyse and add the emotions of the row:
        analyzer = NRCLex(text=row[text_column_name])
        for index, emotion in enumerate(emotions):
            # If not enough emotions in the list, leave column as NaN:
            if len(analyzer.top_emotions) <= index:
                break
            row[emotion] = analyzer.top_emotions[index]

        return row

    df = df.apply(analyze_emotions, axis=1)

    return df

def create_bias(df, text_column_name):
    # Uses nltk vader to calc positive, negative, neutrality and compound columns

    fields = ["pos", "neg", "neu", "compound"]

    # Setup the new columns:
    for field in fields:
        df[field] = np.nan

    sia = SentimentIntensityAnalyzer()

    def analyze_bias(row):
        nonlocal sia
        
        results = sia.polarity_scores(row[text_column_name])

        for field in fields:
            row[field] = results[field]

        return row

    df = df.apply(analyze_bias, axis=1)

    return df

df = pd.DataFrame({"text": ["What a great day", "Today is awful", "I'm so happy", "Looking forward to a great day tomorrow.", "Yesterday was scary"]})

df = create_emotions(df, "text")
df = create_bias(df, "text")
df

     |████████████████████████████████| 396 kB 11.6 MB/s 
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43329 sha256=a3dea2374c74894f295e729a7f55c7583b0c8c02f78e28b146570b395e889056
  Stored in directory: /root/.cache/pip/wheels/af/2c/9c/dfa19d1b65326c520b32850a9311f6d4eda679ac04dba26081
Successfully built nrclex
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,text,emotion_1,emotion_2,emotion_3,pos,neg,neu,compound
0,What a great day,"(fear, 0.0)","(anger, 0.0)","(anticip, 0.0)",0.672,0.000,0.328,0.6249
1,Today is awful,"(fear, 0.2)","(anger, 0.2)","(negative, 0.2)",0.000,0.600,0.400,-0.4588
2,I'm so happy,"(trust, 0.25)","(positive, 0.25)","(joy, 0.25)",0.666,0.000,0.334,0.6115
3,Looking forward to a great day tomorrow.,"(positive, 0.5)","(anticipation, 0.5)",NaN,0.451,0.000,0.549,0.6249
4,Yesterday was scary,"(fear, 0.0)","(anger, 0.0)","(anticip, 0.0)",0.000,0.615,0.385,-0.4939


# Can we observe fake news? / "Most shared covid link by region"

Steps:
- Get regional data
- Clean data (Match the twitter id to 're-hydrate' the data, get the content)
- Follow highly shared links
- Way to determine misinformation?
- Plot globaly
- help: https://link.springer.com/article/10.1007/s11042-021-11621-5

## Fetching data from github

In [ ]:
import pandas as pd
main_data = pd.read_csv("https://media.githubusercontent.com/media/zakstucke/geo-sentiment-analysis/master/covidvaccine.csv?token=ARLQAMR52AXM3H4W7ITZ6O3CGRV6G")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# EDA
main_data.columns

Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet'],
      dtype='object')

In [ ]:
data_head = main_data.head(5000)

In [ ]:
import re
test_url = re.findall('http',str(main_data['text']))

375651

#Geo Sentiment
https://zenodo.org/record/5090588#.YjBMJHrP2Um

# Question 3:



# Question 4:

 # Visualisation

In [ ]:
!pip install tsne
!pip install umap

import tsne
import umap

import numpy as np 
import pandas as pd 

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from scipy.io import loadmat
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
import seaborn as sns

from mpl_toolkits.mplot3d import Axes3D

from tsne import bh_sne
from sklearn.manifold import TSNE

# Ensure up to date version of plotly
!pip install plotly==4.14.3
import plotly.express as px

     |████████████████████████████████| 547 kB 10.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for tsne: filename=tsne-0.3.1-cp37-cp37m-linux_x86_64.whl size=260485 sha256=8cba7fced8c4dac17cb09ac8903272c0c3148692f2b21a6af50cd35ace5c7c30
  Stored in directory: /root/.cache/pip/wheels/b9/a7/9d/d09b0eef66f28be09470e0e18629ae08aed772497b218f84f3
Successfully built tsne
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3565 sha256=5a0134817ed3fab06fe222699810366a4b71fa2b0b7bcedea1f66d0d6ba32252
  Stored in directory: /root/.cache/pip/wheels/65/55/85/945cfb3d67373767e4dc3e9629300a926edde52633df4f0efe
Successfully built umap
     |████████████████████████████████| 13.2 MB 11.2 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=6435001f9458ff80663a856c980d86bb1edda5b72df4a91b1ff20abc07f7d77d
  Stored in directory: /root/.cache/pi

In [ ]:
def scatter2d(X, y):
    fig, ax = plt.subplots(figsize=(15,10))
    ax.scatter(X[:,0], X[:,1], c=y)
    palette = sns.color_palette("husl", len(np.unique(y)))

    for i in range(len(y)):
        ax.text(X[i,0], X[i,1], y[i], color=palette[y[i]-1], fontsize='small')
        
def scatter3d(X, y):
    fig = plt.figure(figsize=(15,10))
    fig.add_subplot(111, projection='3d')
    ax = Axes3D(fig)
    palette = sns.color_palette("husl", len(np.unique(y)))

    ax.mouse_init()
    ax.scatter(X[:,0], X[:,1], X[:,2], c=y)
    for i in range(len(y)):
        ax.text(X[i,0], X[i,1], X[i,2], y[i], color=palette[y[i]-1], fontsize='small')

## MAP VISUALISATION

In [ ]:
fig = px.choropleth(DATASET, # dataset to use
                    locations="Country Code", # column which includes 3 letter country code
                    color="2019", # column which dictates the colour of the map
                    hover_name="Country Name", # column to add to hover information
                    range_color=(0, 30), # range of the colour scale
                    color_continuous_scale="aggrnyl") # colour scale (these can be predefined or you can create your own)
fig.show()

# Start to end data wrangling


In [ ]:
!pip install black==22.1.0
!pip install flake8==3.7.9
!pip install notebook==6.4.3
!pip install ipython==7.31.0
!pip install requests==2.27.1
!pip install pandas==1.4.1
!pip install nltk==3.7
!pip install NRCLex==3.0.0
!pip install geopy==2.2.0

     |████████████████████████████████| 1.4 MB 13.2 MB/s 
     |████████████████████████████████| 97 kB 5.0 MB/s 
     |████████████████████████████████| 843 kB 51.2 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.4 which is incompatible.


     |████████████████████████████████| 69 kB 1.4 MB/s 
     |████████████████████████████████| 51 kB 3.4 MB/s 
     |████████████████████████████████| 59 kB 5.5 MB/s 
  Attempting uninstall: entrypoints
    Found existing installation: entrypoints 0.4
    Uninstalling entrypoints-0.4:
      Successfully uninstalled entrypoints-0.4
     |████████████████████████████████| 9.9 MB 8.6 MB/s 
     |████████████████████████████████| 428 kB 26.2 MB/s 
  Attempting uninstall: tornado
    Found existing installation: tornado 5.1.1
    Uninstalling tornado-5.1.1:
      Successfully uninstalled tornado-5.1.1
  Attempting uninstall: notebook
    Found existing installation: notebook 5.3.1
    Uninstalling notebook-5.3.1:
      Successfully uninstalled notebook-5.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires notebook~=5.3.0; python_version 

     |████████████████████████████████| 792 kB 13.8 MB/s 
     |████████████████████████████████| 380 kB 51.4 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.
google-colab 1.0.0 requires notebook~=5.3.0; python_version >= "3.0", but you have notebook 6.4.3 which is incompatible.
google-colab 1.0.0 requires tor

     |████████████████████████████████| 63 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.
google-colab 1.0.0 requires notebook~=5.3.0; python_version >= "3.0", but you have notebook 6.4.3 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
google-colab 1.0.0 requires tornado~=5.1.0; python_version >= "3.0", but you have tornado 6.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
ERROR: Could not find a version that satisfies the requirement pandas==1.4.1 (from 

     |████████████████████████████████| 118 kB 14.1 MB/s 
  Attempting uninstall: geopy
    Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


In [ ]:
import numpy as np
import nltk
import string


from geopy.geocoders import Nominatim
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nrclex import NRCLex

nltk.download("punkt")
nltk.download("vader_lexicon")

import pandas as pd
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)


def create_emotions(df, text_column_name):
    emotions = ["emotion_1", "emotion_2", "emotion_3"]

    # Add in the new emotion columns:
    for emotion in emotions:
        df[emotion] = np.nan

    def analyze_emotions(row):
        # Analyse and add the emotions of the row:
        analyzer = NRCLex(text=row[text_column_name])
        for index, emotion in enumerate(emotions):
            # If not enough emotions in the list, leave column as NaN:
            if len(analyzer.top_emotions) <= index:
                break
            row[emotion] = analyzer.top_emotions[index]

        return row

    df = df.apply(analyze_emotions, axis=1)

    return df


def create_bias(df, text_column_name):
    # Uses nltk vader to calc positive, negative, neutrality and compound columns

    fields = ["pos", "neg", "neu", "compound"]

    # Setup the new columns:
    for field in fields:
        df[field] = np.nan

    sia = SentimentIntensityAnalyzer()

    def analyze_bias(row):
        nonlocal sia

        results = sia.polarity_scores(row[text_column_name])

        for field in fields:
            row[field] = results[field]

        return row

    df = df.apply(analyze_bias, axis=1)

    return df


def create_cleaned(df, text_column_name):
    fields = ["cleaned_words"]
    for field in fields:
        df[field] = np.nan

    def clean(row):

        text = row[text_column_name]
        # First remove punctuation:
        text = text.translate(str.maketrans("", "", string.punctuation))

        # Split text into words:
        words = text.split()

        # Remove filler words:
        cleaned_words = stripFillerWords(words)

        row["cleaned_words"] = cleaned_words

        return row

    df = df.apply(clean, axis=1)

    return df


def create_geo(df, address_column_name):
    # Adds in the lat long, city and country for the geo information if readable

    fields = ["lat", "long", "city", "country", "country_code"]
    for field in fields:
        df[field] = np.nan

    geolocator = Nominatim(user_agent="bristol_geo_sentiment_analysis_project")

    def _create_geo(row):
        address = row[address_column_name]

        location = geolocator.geocode(address, exactly_one=True, addressdetails=True)
        if location:
            row["lat"] = location.latitude
            row["long"] = location.longitude

            if "address" in location.raw:
                if "city" in location.raw["address"]:
                    row["city"] = location.raw["address"]["city"]
                if "country" in location.raw["address"]:
                    row["country"] = location.raw["address"]["country"]
                if "country_code" in location.raw["address"]:
                    row["country_code"] = location.raw["address"]["country_code"]

        return row

    df = df.apply(_create_geo, axis=1)

    return df

import spacy


def getSingleTweet(data):  # Used for original Main data scraped by TwitterScraper
    # Dictionary has 2 keys
    #       data - for tweet data
    #       includes - for user data
    # Data is made up of a list of tweets, each tweet is its own dictionary
    # This dictionary consists of:
    #                           -id
    #                           -referencedtweets (A list containing a dict for each referenced tweet
    #                           -public_metrics (A dict with: retweet_count,reply_count,like_count,quote_count
    #                           -possibly_sensitive: Bool
    #                           -created_at
    #                           -text
    #                           -lang
    tweetList = data["data"]
    singleTweet = tweetList[0]
    return singleTweet


def getText(data):  # Used for original Main data scraped by TwitterScraper
    tweetList = data["data"]
    allText = []
    counter = 0
    for tweet in tweetList:
        allText.append(str(counter) + ": " + tweet["text"] + "\n\n")
        counter += 1
    return allText


def stripFillerWords(
    words,
):  # Takes a list of words as input and returns list with filler/stop words removed

    # Remove normal filler words (so, was etc):
    nlp = spacy.load("en_core_web_sm")
    result = [word for word in words if word not in nlp.Defaults.stop_words]

    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df = pd.read_csv("https://media.githubusercontent.com/media/zakstucke/geo-sentiment-analysis/master/covidvaccine.csv?token=ARLQAMUQQLR3BHVWWW56WG3CHRRXC", nrows=10)

df = create_cleaned(df, "text")
df = create_bias(df, "text")
df = create_emotions(df, "text")
df = create_geo(df, "user_location")


In [ ]:
print(df)

In [ ]:
filepath = "covidvaccine_geo_emotions"
df.to_csv(filepath)

### Once checkpointed, skip to here

In [ ]:
# import pandas as pd
geo_df = pd.read_csv("covidvaccine_geo_emotions")
geo_df.shape

In [ ]:
# Some issue with this module in default install 
!pip uninstall tornado
!pip install tornado==4.5.3
# Ensure up to date version of plotly
!pip install plotly==4.14.3
import plotly.express as px

Found existing installation: tornado 6.1
Uninstalling tornado-6.1:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/tornado-6.1.dist-info/*
    /usr/local/lib/python3.7/dist-packages/tornado/*
Proceed (y/n)? y
  Successfully uninstalled tornado-6.1
     |████████████████████████████████| 484 kB 12.0 MB/s 
  Created wheel for tornado: filename=tornado-4.5.3-cp37-cp37m-linux_x86_64.whl size=434031 sha256=7af482dede19ff8c2dbac1285c59d0e4ccb548f4d4b600c08cc0c8cd19adb00b
  Stored in directory: /root/.cache/pip/wheels/a2/45/43/36ec7a893e16c1212a6b1505ded0a2d73cf8e863a0227c8e04
Successfully built tornado
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.4.3 requires tornado>=6.1, but you have tornado 4.5.3 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.0 which is incompatible.
google-colab 1.0.0 requires

In [ ]:
geo_df.columns

Index(['Unnamed: 0', 'user_name', 'user_location', 'user_description',
       'user_created', 'user_followers', 'user_friends', 'user_favourites',
       'user_verified', 'date', 'text', 'hashtags', 'source', 'is_retweet',
       'cleaned_words', 'pos', 'neg', 'neu', 'compound', 'emotion_1',
       'emotion_2', 'emotion_3', 'lat', 'long', 'city', 'country',
       'country_code'],
      dtype='object')

In [ ]:
# Check for current country codes, looks like geo_df is tiny for some reason
cc_present = geo_df["country_code"].unique()

In [ ]:
# geo_df["len"] = len(geo_df["emotion_1"])
# geo_df
import regex as re

# f = lambda x: re.search('\([a-zA-Z]*,',x)
# geo_df["main_emo"] = geo_df.apply(f)

# f = lambda x: re.search('\([a-zA-Z]*,',x)
# geo_df["main_emo"] = geo_df.apply(re.search('\([a-zA-Z]*,',geo_df["emotion_1"]))


def extract_emotion(df, text_column_name):
    
    def emotion(row):

        text = row[text_column_name]
        # First remove punctuation:
    #    text = re.search('\([a-zA-Z]*,',text)

        # Split text into words:
        # emo = text.split(",")
        emo = text[0]
        
        # Take out () brackets
        emo = emo.strip("()")
        
        row["extracted_emotion"] = emo

        return row

    df = df.apply(emotion, axis=1)

    return df


def extract_url(df, text_column_name):  
    
    def url(row):

        text = row[text_column_name]
        
        # Look for urls
        url = re.findall("https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()!@:%_\+.~#?&\/\/=]*)", text)
        
        row["extracted_url"] = url

        return row

    df = df.apply(url, axis=1)

    return df

df2 = extract_emotion(df, "emotion_1")
df2 = extract_url(df2, "text")


In [ ]:
df2

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,cleaned_words,pos,neg,neu,compound,emotion_1,emotion_2,emotion_3,lat,long,city,country,country_code,extracted_emotion,extracted_url
0,MyNewsNE,Assam,MyNewsNE a dedicated multi-lingual media house...,24-05-2020 10:18,64.0,11.0,110.0,False,18-08-2020 12:55,Australia to Manufacture Covid-19 Vaccine and ...,['CovidVaccine'],Twitter Web App,False,"[Australia, Manufacture, Covid19, Vaccine, Cit...",0.148,0.000,0.852,0.5106,"(fear, 0.0)","(anger, 0.0)","(anticip, 0.0)",26.407384,93.255130,NaN,India,in,fear,[]
1,Shubham Gupta,NaN,I will tell about all experiences of my life f...,14-08-2020 16:42,1.0,17.0,0.0,False,18-08-2020 12:55,#CoronavirusVaccine #CoronaVaccine #CovidVacci...,"['CoronavirusVaccine', 'CoronaVaccine', 'Covid...",Twitter for Android,False,"[CoronavirusVaccine, CoronaVaccine, CovidVacci...",0.285,0.000,0.715,0.4927,"(trust, 0.2)","(surprise, 0.2)","(positive, 0.2)",46.314475,11.048029,Ville d'Anaunia,Italia,it,trust,[]
2,Journal of Infectiology,NaN,Journal of Infectiology (ISSN 2689-9981) is ac...,14-12-2017 07:07,143.0,566.0,8.0,False,18-08-2020 12:46,Deaths due to COVID-19 in Affected Countries\n...,NaN,Twitter Web App,False,"[Deaths, COVID19, Affected, Countries, Read, M...",0.000,0.118,0.882,-0.1531,"(fear, 0.0)","(anger, 0.0)","(anticip, 0.0)",46.314475,11.048029,Ville d'Anaunia,Italia,it,fear,[]
3,Zane,NaN,Fresher than you.,18-09-2019 11:01,29.0,25.0,620.0,False,18-08-2020 12:45,@Team_Subhashree @subhashreesotwe @iamrajchoco...,NaN,Twitter for Android,False,"[TeamSubhashree, subhashreesotwe, iamrajchoco,...",0.209,0.000,0.791,0.4404,"(trust, 0.3333333333333333)","(positive, 0.3333333333333333)","(joy, 0.3333333333333333)",46.314475,11.048029,Ville d'Anaunia,Italia,it,trust,[]
4,Ann-Maree O’Connor,"Adelaide, South Australia",Retired university administrator. Melburnian b...,24-01-2013 14:53,83.0,497.0,10737.0,False,18-08-2020 12:45,@michellegrattan @ConversationEDU This is what...,NaN,Twitter Web App,False,"[michellegrattan, ConversationEDU, This, passe...",0.000,0.000,1.000,0.0000,"(trust, 1.0)",NaN,NaN,-34.928181,138.599931,Adelaide,Australia,au,trust,[]
5,Raunak Scherbatsky DankWorth,NaN,Neuro surgeon + Diagnostician.👨‍⚕️\na good phy...,03-08-2020 13:39,3.0,27.0,918.0,False,18-08-2020 12:44,The Multi-system Inflammatory Syndrome-Childre...,"['COVID19', 'COVID19India']",Twitter for Android,False,"[The, Multisystem, Inflammatory, SyndromeChild...",0.000,0.000,1.000,0.0000,"(fear, 0.2)","(anger, 0.2)","(negative, 0.2)",46.314475,11.048029,Ville d'Anaunia,Italia,it,fear,[]
6,Rajesh Tadepalli,"Hyderabad, India",Growth Initiatives @tech_mahindra | AIMer | Br...,07-05-2013 03:57,918.0,2561.0,31837.0,False,18-08-2020 12:34,@PrivilRodrigues @yatish57 @deepkaranahuja @sh...,NaN,Twitter for Android,False,"[PrivilRodrigues, yatish57, deepkaranahuja, sh...",0.000,0.000,1.000,0.0000,"(fear, 0.0)","(anger, 0.0)","(anticip, 0.0)",17.360589,78.474061,Hyderabad,India,in,fear,[]
7,AKisASocialisolationist wash yer damn hands,The Great Pacific Northwest,"Iconoclast, cat person, soccer fan, textile & ...",07-02-2015 07:24,2321.0,3236.0,264351.0,False,18-08-2020 12:30,"@MSNBC Well, let’s qualify that: would anyone ...",['CovidVaccine'],Twitter for iPhone,False,"[MSNBC, Well, let’s, qualify, party, vaccine, ...",0.202,0.000,0.798,0.5859,"(positive, 0.5)","(anticipation, 0.5)",NaN,NaN,NaN,NaN,NaN,NaN,positive,[]
8,Dr. Joseph Santoro,"Washington, DC 20009","Neuro PhD, #Innovator, #Technologist, #Startup...",17-01-2009 21:10,19091.0,20986.0,128119.0,False,18-08-2020 12:15,"Most countries, without the ability to make #V...",['Vaccines'],Hootsuite Inc.,False,"[Most, countries, ability, Vaccines, locally, ...",0.098,0.195,0.707,-0.3532,"(fear, 0.3333333333333333)","(positive, 0.3333333333333333)","(negative, 0.3333333333333333)",38.895037,-77.036543,Washington,United States,us,fear,[]
9,VUMC 

In [ ]:
# df2.columns
group_df = df2.groupby("country_code")["country_code", "extracted_emotion"].agg(pd.Series.mode)
# mode_emotion = geo_df.groupby(["country_code", "emotion_1"]).count()

group_df.columns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



Index(['country_code', 'extracted_emotion'], dtype='object')

In [ ]:
fig = px.choropleth(group_df, # dataset to use
                    locations=["Uk","Poland","Brazil"], # column which includes 3 letter country code
                    locationmode = "country names",
                    color=[97,6,170], # column which dictates the colour of the map
                    #hover_name=["au","it"], # column to add to hover information
                    range_color=(0, 200), #,# range of the colour scale
                    color_continuous_scale=["red","blue"] # colour scale (these can be predefined or you can create your own)
)
fig.show()
# "#ff009f","cyan"

('#ff009f', 'cyan')

# Vaccine chloropleth

In [ ]:
# imports
import pandas as pd
import plotly.express as px

# Data IN
vaccine_data = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
df = pd.read_csv(vaccine_data)


# load in processed data
pd.options.mode.chained_assignment = None  # default='warn' this is needed for false positives warnings on reassigning pd dataframe

#CASES DATA FROM CSV
WHOfile = "data/WHO_covid_data_by_COUNTRY.csv"
WHOdf = pd.read_csv(WHOfile, usecols=["iso_code", "date", "new_cases"])

#TWEET DATA FROM PREPROCESSED DF
with open("./processed_data/final_df.json", "r") as file:
    COVIDdf = pd.read_json(json.load(file))

In [ ]:
# Features for chloropleth
features = df
print(features.shape)

col = "people_vaccinated"

# Delete rows where case numbers are zero
# This deletion is completed by "selecting" rows where case numbers are non zero
features = features.loc[features[col] != 0]
features = features.loc[features["location"] != "World"].dropna()
print(features.shape)

max_total = features[col].max()
min_total = features[col].min()

# Delete rows where there have been no cases in 24 hours AND no cases in 7 days
# Note that you must put each condition within parenthesis
# data = data.loc[(data["deaths_7_days"] > 0) & (data["deaths_24_hours"] > 0)]
# data.shape

(92673, 16)
(18131, 16)


In [ ]:
# Chloropleth
fig = px.choropleth(features, # dataset to use
                    locations="iso_code", # column which includes 3 letter country code
                    locationmode = "ISO-3",
                    color=col, # column which dictates the colour of the map
                    hover_name="location", # column to add to hover information
                    range_color=(min_total, max_total), #,# range of the colour scale
                    color_continuous_scale=["red","blue"] # colour scale (these can be predefined or you can create your own)
)
fig.show()

## People vaccinated per 100

In [ ]:
# Features for chloropleth
features = df
print(features.shape)

col = "people_fully_vaccinated_per_hundred"

# Delete rows where case numbers are zero
# features = features.loc[features[col] != 0]

# Delete "World" entries, these cause anomalies 
features = features.loc[features["location"] != "World"]


def featurise(df, text_column_name):   
  def f(row):
    if ("OWID" not in row[text_column_name]):
      return row
    else:
      row[text_column_name] = None
      return row

  df = df.apply(f, axis=1)
  return df

features = featurise(features, "location")
# features = features.loc[features["iso_code"].find("OWID")]

features = features.loc[features["location"] != "Gibraltar"].dropna()
print(features.shape)

# Delete rows where there have been no cases in 24 hours AND no cases in 7 days
# Note that you must put each condition within parenthesis
# data = data.loc[(data["deaths_7_days"] > 0) & (data["deaths_24_hours"] > 0)]
# data.shape

(92673, 16)
(18060, 16)
92.6


,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million,daily_people_vaccinated,daily_people_vaccinated_per_hundred
location,,,,,,,,,,,,,,,
Luxembourg,LUX,2021-04-13,141441.0,103569.0,37870.0,2.0,6269.0,4091.0,22.28,16.31,5.97,0.00,6444.0,2460.0,0.388
Low income,OWID_LIC,2022-03-27,136775532.0,96461180.0,74837038.0,3392787.0,2625674.0,2057654.0,20.56,14.50,11.25,0.51,3094.0,1339952.0,0.201
Kenya,KEN,2022-02-13,15236700.0,8054405.0,6958610.0,223685.0,282646.0,233947.0,27.71,14.65,12.66,0.41,4255.0,128174.0,0.233
Africa,OWID_AFR,2022-04-03,450723336.0,278980217.0,208759663.0,18462770.0,4169685.0,2949068.0,32.82,20.31,15.20,1.34,2147.0,1791007.0,0.130
Libya,LBY,2022-03-24,3415834.0,2201314.0,1125758.0,89762.0,15033.0,9419.0,49.09,31.63,16.18,1.29,1354.0,4133.0,0.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
United Arab Emirates,ARE,2021-11-25,21757061.0,9881456.0,8914179.0,2961426.0,130177.0,79801.0,217.76,98.90,89.22,29.64,7987.0,36739.0,0.368
Malta,MLT,2022-04-02,1257537.0,474022.0,466726.0,351083.0,10656.0,8191.0,243.66,91.85,90.43,68.03,15871.0,873.0,0.169
Chile,CHL,2022-03-29,50672753.0,17872888.0,17377173.0,15997203.0,324398.0,223961.0,263.75,93.03,90.45,83.27,11657.0,81900.0,0.426


In [ ]:
# Last known highest number of people fully vaccinated
max_total = features[col].max()
min_total = features[col].min()

agg_max = features.groupby("location").agg("max")
agg_max = agg_max.sort_values("people_fully_vaccinated_per_hundred")
agg_max

# Chloropleth
fig = px.choropleth(agg_max, # dataset to use
                    locations="iso_code", # column which includes 3 letter country code
                    locationmode = "ISO-3",
                    color=col, # column which dictates the colour of the map
                  #  hover_name="location", # column to add to hover information
                    range_color=(min_total, max_total), #,# range of the colour scale
                    color_continuous_scale=["red","blue"] # colour scale (these can be predefined or you can create your own)
)
fig.show()

In [ ]:
# Forwards fill the dataframe (assuming no reporting)

c

unique dates = 461
unique countries = 107


In [ ]:

b = pd.merge(a, features,  how='left', left_on=['locations','date'], right_on = ['location','date'])
b = b.sort_values("date")
b = b.fillna(method = 'ffill') #.replace(to_replace=0, method='ffill')
b["people_fully_vaccinated_per_hundred"]


# extra_dates = pd.concat([pd.DataFrame([i], columns=['date']) for i in dates], ignore_index=True)
# extra_dates.shape
# over_time = features.sort_values("date")
# # features.columns
# over_time[col].replace(to_replace=0, method='ffill')
# over_time


# Chloropleth
fig = px.choropleth(b, # dataset to use
                    locations="iso_code", # column which includes 3 letter country code
                    locationmode = "ISO-3",
                    animation_frame = "date",
                    color=col, # column which dictates the colour of the map
                    hover_name="location", # column to add to hover information
                    range_color=(min_total, max_total), #,# range of the colour scale
                    color_continuous_scale=["red","blue"] # colour scale (these can be predefined or you can create your own)
)
fig.show()

In [ ]:
len(b.loc[b["location"] == "Africa"]["people_fully_vaccinated_per_hundred"].tolist())

241

In [ ]:
print(b.shape)

(49327, 17)


In [1]:
!pip install spacy_langdetect
import spacy
from spacy_langdetect import LanguageDetector

import pandas as pd
from collections import Counter
import numpy as np
import string

COVIDdf = pd.read_csv("https://media.githubusercontent.com/media/zakstucke/geo-sentiment-analysis/master/processed_data/final_csv.txt?token=ARLQAMT3LSSZZKJKR4Z2YNDCME4DI")
#text,retweet_count,like_count,created_at,lang,author_location,emotion_1,emotion_2,emotion_3,pos,neg,neu,compound,cleaned_words,iso_code,country,city,lat,lng
#COVIDdf.rename(columns={'created_at': 'date'}, inplace=True)

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe(LanguageDetector(), name="language_detector", last=True)
stop_words = nlp.Defaults.stop_words

     |████████████████████████████████| 998 kB 8.0 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993431 sha256=e369c8e2e855b8d63f78e36f350e7b84046591ba26b15be2d07e7f774823ed61
  Stored in directory: /root/.cache/pip/wheels/89/79/3b/9885ae7f4308f73c514f96d8574d40d7d8173a27731b674013
Successfully built langdetect


HTTPError: ignored

In [ ]:
def splitByMonth(df, col):
    df = df[df[col].notna()]
    dates = pd.to_datetime(df[col])
    df[col] = dates
    df = df.set_index(col)
    strText = []
    for group_name, df_group in df.groupby(pd.Grouper(freq='M')):
        strText.append(df_group)
    return strText


def getPopularWords(df, overall):#Input a list of strings and amount of popular words needed, returns a list of popular words
      
    # # Remove filler words (can only currently do with english):
    # if row["lang"] == "en":
    #     row["cleaned_words"] = stripFillerWords(words)
    # else:
    #     row["cleaned_words"] = words
    # return row
  
  inOne = ""
  for message in df.text:
    doc = nlp(message)
    # print(doc._.language['language'])
    if (doc._.language['language'] == 'en'):
      #message = message.translate(str.maketrans("", "", string.punctuation))
      inOne += message.lower() + " "
  justWords = inOne.split()#Splitting into words

  justWords = stripFillerWords(justWords) #ADDED
  popularWords = Counter(justWords)
  popularOverall = overall + popularWords
  return popularOverall #Looked into source code of counter and appears to be O(n) time complexity


def stripFillerWords(
    words,
):  # Takes a list of words as input and returns list with filler/stop words removed
    # Remove normal filler words (so, was etc):
    result = [word for word in words if word not in stop_words]

    # print("English word: " + next(iter(englishWords)))
    # print(word + " is English? " + nlp.vocab.strings[word])

    return result



In [ ]:


#############

# dfs = splitByMonth(COVIDdf, "created_at")

# overall = Counter("")
# # df_new = pd.dataFrame(list(overall))
# for i,df in enumerate(dfs):
#   overall = getPopularWords(df, overall)
#   df[i] = overall
#   print(overall.most_common()[0:30])


######M########
dfs = splitByMonth(COVIDdf, "created_at")

overall = Counter("")
df_new = pd.DataFrame(list(overall))
for i,df in enumerate(dfs[0:6]):
  overall = getPopularWords(df, overall)
  df_new[i] = overall.most_common()
  # print(overall.most_common()[0:30])



ValueError: ignored

In [ ]:
# Merges obj2 into obj1
def merge_dicts(obj1, obj2):
    assert type(obj1) == dict and type(obj2) == dict

    for key in obj2:
        if key not in obj1:
            obj1[key] = obj2[key]
        else:
            assert type(obj1[key]) == type(
                obj2[key]
            ), "Why do the types not match if they have the same key? Key: {}".format(key)

            if type(obj1[key]) == dict:
                obj1[key] = merge_dicts(obj1[key], obj2[key])
            elif type(obj1[key]) == list:
                obj1[key] += obj2[key]
            else:
                raise Exception(
                    "Obj1 and 2 have key values which aren't list or dict that are trying to be merged, key: {}".format(
                        key
                    )
                )

    return obj1